In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchinfo import summary

import os
import glob
import time
import datetime as dt
from netCDF4 import Dataset as nc_Dataset
from netCDF4 import date2num, num2date
import pandas as pd
import numpy as np
import math
import xarray as xr
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.markers import MarkerStyle
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import mpl_scatter_density
from mpl_scatter_density import ScatterDensityArtist
from matplotlib.colors import LinearSegmentedColormap
from sklearn.linear_model import LinearRegression

from FunctionsAndClasses.HRRR_URMA_Datasets_AllVars import *
from FunctionsAndClasses.SR_UNet_simple import *
from FunctionsAndClasses.DefineModelAttributes import *
from FunctionsAndClasses.StatObjectConstructor import *
from FunctionsAndClasses.utils import *
from FunctionsAndClasses.CONSTANTS import *

from FunctionsAndClasses.Dataset_TESTING import *

In [2]:
HRRR_PATH = f"/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/"
URMA_PATH = f"/data1/projects/RTMA/alex.schein/URMA_train_test/"

In [3]:
var_name = 't2m'

In [4]:
test_model_attrs = DefineModelAttributes(is_train=False,
                                         with_terrains=['diff'],
                                         predictor_vars=['t2m'],
                                         target_vars=['t2m'],
                                         BATCH_SIZE=18,
                                         NUM_EPOCHS=10)

In [5]:
NUM_GPUS_TO_USE=3
MULTIGPU_BATCH_SIZE = test_model_attrs.BATCH_SIZE*NUM_GPUS_TO_USE

In [6]:
ds_test = Dataset_TEST(load_xr_into_memory=False, with_terrain_diff=True)

Ignoring index file '/data1/projects/RTMA/alex.schein/Regridded_HRRR_train_test/train_hrrr_alltimes_t2m_f01.grib2.5b7b6.idx' incompatible with GRIB file
Ignoring index file '/data1/projects/RTMA/alex.schein/URMA_train_test/train_urma_alltimes_t2m.grib2.5b7b6.idx' incompatible with GRIB file
Ignoring index file '/data1/projects/RTMA/alex.schein/Terrain_Maps/terrain_subset_HRRR_2p5km.grib2.5b7b6.idx' incompatible with GRIB file
Ignoring index file '/data1/projects/RTMA/alex.schein/Terrain_Maps/terrain_subset_URMA_2p5km.grib2.5b7b6.idx' incompatible with GRIB file


In [7]:
dl_test = DataLoader(ds_test, 
                     batch_size=MULTIGPU_BATCH_SIZE, 
                     shuffle=True, 
                     num_workers=4*NUM_GPUS_TO_USE, 
                     pin_memory=True)

In [11]:
model = SR_UNet_simple(n_channels_in=2, 
                        n_channels_out=1)
device = torch.device("cuda")
# model = nn.DataParallel(model, device_ids=[i for i in range(NUM_GPUS_TO_USE)])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=[0.5,0.999])

loss_function = torch.nn.L1Loss()

RuntimeError: CUDA driver initialization failed, you might not have a CUDA gpu.

In [12]:
torch.cuda.is_available()

False

In [ ]:
# model.train()

In [ ]:
# lowest_loss = 999
# for epoch in range(1,test_model_attrs.NUM_EPOCHS+1):
#     epoch_loss = 0.0
#     start = time.time()
#     for i, (inputs,labels) in enumerate(dl_test):    
#         start_dl = time.time()
#         print(f"Starting batch {i+1}/{len(dl_test)} in epoch {epoch}")
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()

#         outputs = model(inputs.float()) #weird datatype mismatching... for some reason it's seeing HRRR data as double
#         loss = loss_function(outputs,labels)
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

#         print(f"Done with batch {i+1}/{len(dl_test)}. Time taken = {time.time()-start_dl:.1f} sec. Running loss = {epoch_loss/(i+1):.5f}")
        
#     end = time.time() 
    
#     if epoch_loss <= lowest_loss: #only save models that have lower loss than previous best
#         lowest_loss = epoch_loss

#     print(f"End of epoch {epoch} | Average loss for epoch = {epoch_loss/len(dl_test):.5f} | Time for epoch = {end-start:.1f} sec")

In [ ]:
# TRAINED_MODEL_SAVEPATH = "/scratch/RTMA/alex.schein/CNN_Main/Trained_models"
# torch.save(model.module.state_dict(), f"{TRAINED_MODEL_SAVEPATH}/{test_model_attrs.savename}.pt")

In [ ]:
###############

In [ ]:
IDX = 3000

In [ ]:
pred, targ = ds_test[IDX]
pred = pred[np.newaxis,:] 
pred_gpu = torch.from_numpy(pred).cuda(device)

with torch.no_grad():
    model_output = model(pred_gpu.float())
    model_output = model_output.cpu().numpy()

In [ ]:
C = CONSTANTS()

hrrr_mean = C.hrrr_means_dict['train'][f"{var_name}"]
hrrr_std = C.hrrr_stddevs_dict['train'][f"{var_name}"]

urma_mean = C.urma_means_dict['train'][f"{var_name}"]
urma_std = C.urma_stddevs_dict['train'][f"{var_name}"]

In [ ]:
np.shape(pred.squeeze()[0])

In [ ]:
pred_unnormed = hrrr_std*pred.squeeze()[0] + hrrr_mean
model_output_unnormed = urma_std*model_output + urma_mean
targ_unnormed = urma_std*targ + urma_mean

In [ ]:
plot_predictor_output_truth_error(pred_unnormed, 
                                  model_output_unnormed, 
                                  targ_unnormed, 
                                  title=test_model_attrs.savename, 
                                  date_str=f"IDX {IDX}",
                                  to_save=False,
                                  save_dir=f"/scratch/RTMA/alex.schein/", 
                                  fig_savename=f"IDX{IDX}_{test_model_attrs.savename}.png")

In [ ]:
plot_predictor_output_truth_error(pred_unnormed, pred_unnormed, targ_unnormed, title="No model", date_str=f"IDX {IDX}")